# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [3]:
from azureml.core import Workspace, Experiment

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [4]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'student-math-experiment'

experiment=Experiment(ws, experiment_name)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [13]:
from azureml.core import Dataset

# Load dataset from Azure ML workspace
df = Dataset.get_by_name(workspace=ws, name="student-math")


In [21]:
from azureml.train.automl import AutoMLConfig

# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 15,  
    "primary_metric": "normalized_mean_absolute_error",  
    "n_cross_validations": 5, 
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    task="regression",  
    training_data=df,  
    label_column_name="G3", 
    compute_target="notebook-compute", 
    enable_early_stopping=True,
    featurization="auto",  
    **automl_settings
) 

In [22]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
student-math-experiment,AutoML_7983c695-9158-4ea8-9c33-d5dc094e382c,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [23]:
from azureml.widgets import RunDetails

# Show run details in the notebook
RunDetails(remote_run).show()

2025-03-20 14:17:59.932524: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-20 14:18:01.612481: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-20 14:18:02.112316: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-20 14:18:05.853115: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-20 14:18:13.465092: W tensorflow/comp

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [24]:
best_run, best_model = remote_run.get_output()

In [25]:
import joblib

#TODO: Save the best model
print("Best Run ID:", best_run.id)
print("Best Model Properties:")
for key, value in best_run.get_metrics().items():
    print(f"{key}: {value}")


model_filename = "best_automl_model.pkl"
joblib.dump(best_model, model_filename)

Best Run ID: AutoML_7983c695-9158-4ea8-9c33-d5dc094e382c_30
Best Model Properties:
mean_absolute_error: 1.2260418870909464
normalized_mean_absolute_error: 0.061302094354547323
root_mean_squared_error: 1.8848238694943518
root_mean_squared_log_error: 0.467761504956451
normalized_median_absolute_error: 0.03982582688961778
normalized_root_mean_squared_log_error: 0.15364035395522496
normalized_root_mean_squared_error: 0.0942411934747176
explained_variance: 0.8230660143030084
r2_score: 0.8221452846270634
spearman_correlation: 0.940696493743275
mean_absolute_percentage_error: 9.793170407317607
median_absolute_error: 0.7965165377923554
residuals: aml://artifactId/ExperimentRun/dcid.AutoML_7983c695-9158-4ea8-9c33-d5dc094e382c_30/residuals
predicted_true: aml://artifactId/ExperimentRun/dcid.AutoML_7983c695-9158-4ea8-9c33-d5dc094e382c_30/predicted_true


['best_automl_model.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [5]:
from azureml.core.model import Model

# Register the best model
model = Model.register(
    workspace=ws,
    model_name="student-performance-model",  
    model_path="best_automl_model.pkl",
    description="AutoML trained model for predicting student performance"
)

Registering model student-performance-model


In [9]:
from azureml.core.environment import Environment

env = Environment(name="student-performance-env")
env.python.conda_dependencies.add_pip_package("azureml-defaults")
env.python.conda_dependencies.add_pip_package("scikit-learn") 
env.python.conda_dependencies.add_pip_package("pandas")
env.python.conda_dependencies.add_pip_package("joblib")
env.python.conda_dependencies.add_pip_package("numpy")
env.python.conda_dependencies.add_pip_package("azureml-core")

In [10]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice


service_name = 'student-performance-predictor'

inference_config = InferenceConfig(entry_script='score.py', environment=env)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                        name=service_name,
                        models=[model],
                        inference_config=inference_config,
                        deployment_config=aci_config,
                        overwrite=True)
service.wait_for_deployment(show_output=True)

/tmp/ipykernel_2930/1537714338.py:10: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2025-03-20 15:25:05+00:00 Creating Container Registry if not exists..
2025-03-20 15:35:05+00:00 Registering the environment.
2025-03-20 15:35:08+00:00 Building image..
2025-03-20 15:43:23+00:00 Generating deployment configuration.
2025-03-20 15:43:24+00:00 Submitting deployment to compute..
2025-03-20 15:43:36+00:00 Checking the status of deployment student-performance-predictor..
2025-03-20 15:45:03+00:00 Checking the status of inference endpoint student-performance-predictor.

KeyboardInterrupt: 

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
input_data = {
    "data": [
        {
            "school": "GP", "sex": "F", "age": 16, "address": "R",
            "famsize": "LE3", "Pstatus": "T", "Medu": 2, "Fedu": 3,
            "Mjob": "other", "Fjob": "teacher", "reason": "home",
            "guardian": "father", "traveltime": 3, "studytime": 3,
            "failures": 2, "schoolsup": "yes", "famsup": "yes",
            "paid": "yes", "activities": "no", "nursery": "yes",
            "higher": "no", "internet": "yes", "romantic": "no",
            "famrel": 1, "freetime": 2, "goout": 4, "Dalc": 5,
            "Walc": 5, "health": 1, "absences": 8, "G1": 15, "G2": 20
        },
        {
            "school": "GP", "sex": "M", "age": 16, "address": "U",
            "famsize": "GT3", "Pstatus": "A", "Medu": 2, "Fedu": 0,
            "Mjob": "health", "Fjob": "health", "reason": "home",
            "guardian": "other", "traveltime": 3, "studytime": 2,
            "failures": 1, "schoolsup": "no", "famsup": "yes",
            "paid": "yes", "activities": "yes", "nursery": "no",
            "higher": "no", "internet": "yes", "romantic": "no",
            "famrel": 1, "freetime": 4, "goout": 4, "Dalc": 1,
            "Walc": 2, "health": 1, "absences": 18, "G1": 8, "G2": 10
        },
        {
            "school": "GP", "sex": "M", "age": 20, "address": "U",
            "famsize": "LE3", "Pstatus": "T", "Medu": 2, "Fedu": 0,
            "Mjob": "teacher", "Fjob": "other", "reason": "reputation",
            "guardian": "other", "traveltime": 3, "studytime": 1,
            "failures": 0, "schoolsup": "no", "famsup": "yes",
            "paid": "yes", "activities": "no", "nursery": "yes",
            "higher": "yes", "internet": "no", "romantic": "yes",
            "famrel": 1, "freetime": 2, "goout": 2, "Dalc": 3,
            "Walc": 2, "health": 1, "absences": 11, "G1": 17, "G2": 13
        }
    ]
}

In [ ]:
import requests
import json

# Replace with your actual endpoint URL
endpoint_url = "https://your-endpoint-url.azurewebsites.net/score"

# If your service requires authentication, include the key
api_key = "your-api-key-if-required"  # Replace with actual key if needed
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}" if api_key else None
}

# Send request to the web service
response = requests.post(endpoint_url, data=json.dumps(input_data), headers=headers)

# Print response
print("Prediction Response:", response.json())

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
